In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

class ACOBinPacking:
    def __init__(self, item_sizes, capacity, num_ants=20, max_iter=100, alpha=1.0, beta=2.0, rho=0.1, Q=1.0):
        self.item_sizes = np.array(item_sizes, dtype=float)
        self.n = len(item_sizes)
        self.C = float(capacity)
        self.num_ants = num_ants
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q

        # Sort items in decreasing order (like FFD)
        self.sorted_indices = np.argsort(-self.item_sizes)
        self.item_sizes = self.item_sizes[self.sorted_indices]

        # Pheromone matrix (symmetric)
        self.tau = np.ones((self.n, self.n)) * 0.1

        # Best solution tracking
        self.best_num_bins = self.n  # Worst case
        self.best_unused = float('inf')
        self.best_packing = None
        self.convergence = []

    def construct_solution(self):
        bins = []  # List of lists: each sublist is item indices in that bin
        bin_loads = []
        for i in range(self.n):
            probs = []
            candidates = []
            # Existing bins
            for j in range(len(bins)):
                remaining = self.C - bin_loads[j]
                if remaining >= self.item_sizes[i]:
                    # Average pheromone with items in bin
                    tau_avg = np.mean([self.tau[i, k] for k in bins[j]]) if bins[j] else 0.1
                    # Heuristic: prefer tight fit (small remaining after placement)
                    after_remaining = remaining - self.item_sizes[i]
                    eta = 1 / (after_remaining + 1)  # Higher if tighter
                    attr = (tau_avg ** self.alpha) * (eta ** self.beta)
                    probs.append(attr)
                    candidates.append(j)

            # Option to open new bin
            tau_new = 0.01  # Low to discourage
            eta_new = 0.01  # Low to discourage
            attr_new = (tau_new ** self.alpha) * (eta_new ** self.beta)
            probs.append(attr_new)
            candidates.append(-1)

            # If no existing bin can fit, force new bin
            if not candidates[:-1]:
                selected = -1
            else:
                # Normalize probabilities
                total = sum(probs)
                if total == 0:
                    probs = [1.0 / len(probs)] * len(probs)
                else:
                    probs = [p / total for p in probs]
                # Roulette wheel selection
                choice = np.random.choice(range(len(probs)), p=probs)
                selected = candidates[choice]

            if selected == -1:
                # Open new bin
                bins.append([i])
                bin_loads.append(self.item_sizes[i])
            else:
                # Add to existing bin
                bins[selected].append(i)
                bin_loads[selected] += self.item_sizes[i]

        return bins, bin_loads

    def run(self):
        start_time = time.time()
        for iteration in range(self.max_iter):
            ant_solutions = []
            for ant in range(self.num_ants):
                packing, loads = self.construct_solution()
                num_bins = len(packing)
                unused = sum(self.C - load for load in loads if load > 0)
                ant_solutions.append((packing, num_bins, unused))

            # Sort by number of bins, then unused space
            ant_solutions.sort(key=lambda x: (x[1], x[2]))

            # Update best
            if ant_solutions[0][1] < self.best_num_bins or \
               (ant_solutions[0][1] == self.best_num_bins and ant_solutions[0][2] < self.best_unused):
                self.best_packing = ant_solutions[0][0]
                self.best_num_bins = ant_solutions[0][1]
                self.best_unused = ant_solutions[0][2]

            # Pheromone update using global best
            self.tau *= (1 - self.rho)  # Evaporation
            delta = self.Q / self.best_num_bins
            for bin_items in self.best_packing:
                for idx1 in range(len(bin_items)):
                    for idx2 in range(idx1 + 1, len(bin_items)):
                        i = bin_items[idx1]
                        k = bin_items[idx2]
                        self.tau[i, k] += delta
                        self.tau[k, i] += delta  # Keep symmetric

            self.convergence.append(self.best_num_bins)

        runtime = time.time() - start_time
        return self.best_num_bins, self.best_unused, runtime, self.convergence

    def plot_convergence(self, filename=None):
        plt.plot(self.convergence)
        plt.xlabel('Iteration')
        plt.ylabel('Best Number of Bins')
        plt.title('Convergence Plot')
        if filename:
            plt.savefig(filename)
            plt.close()
        else:
            plt.show()

    def plot_load_distribution(self, filename=None):
        if self.best_packing is None:
            print("Run the algorithm first.")
            return
        loads = [sum(self.item_sizes[items] for items in bin_) for bin_ in self.best_packing]
        plt.bar(range(len(loads)), loads)
        plt.ylim(0, self.C)
        plt.xlabel('Bin Index')
        plt.ylabel('Load')
        plt.title('Load Distribution Across Bins')
        if filename:
            plt.savefig(filename)
            plt.close()
        else:
            plt.show()

    def plot_item_size_distribution(self, filename=None):
        plt.hist(self.item_sizes, bins=20)
        plt.xlabel('Item Size')
        plt.ylabel('Frequency')
        plt.title('Item Size Distribution')
        if filename:
            plt.savefig(filename)
            plt.close()
        else:
            plt.show()

def load_binpack_file(filename, instance_idx=0, is_float=False):
    with open(filename, 'r') as f:
        lines = f.readlines()
    line_idx = 0
    P = int(lines[line_idx].strip())
    line_idx += 1
    for p in range(instance_idx + 1):
        identifier = lines[line_idx].strip()
        line_idx += 1
        parts = lines[line_idx].strip().split()
        if is_float:
            capacity = float(parts[0])
            n = int(parts[1])
            best_known = int(parts[2])
        else:
            capacity = int(parts[0])
            n = int(parts[1])
            best_known = int(parts[2])
        line_idx += 1
        sizes = []
        for _ in range(n):
            if is_float:
                sizes.append(float(lines[line_idx].strip()))
            else:
                sizes.append(int(lines[line_idx].strip()))
            line_idx += 1
    return sizes, capacity, best_known, identifier

# Assuming files are in 'data/1Dimensional/' directory
data_dir = 'data/1Dimensional/'
os.makedirs('plots', exist_ok=True)

instances = []
for i in range(1, 5):
    filename = os.path.join(data_dir, f'binpack{i}.txt')
    sizes, C, best_known, name = load_binpack_file(filename, 0, is_float=False)
    instances.append({"name": name, "capacity": C, "best_known": best_known, "sizes": sizes})

for i in range(5, 9):
    filename = os.path.join(data_dir, f'binpack{i}.txt')
    sizes, C, best_known, name = load_binpack_file(filename, 0, is_float=True)
    instances.append({"name": name, "capacity": C, "best_known": best_known, "sizes": sizes})

results = []
for instance in instances:
    aco = ACOBinPacking(instance["sizes"], instance["capacity"])
    num_bins, unused, runtime, convergence = aco.run()
    results.append({
        "Instance": instance["name"],
        "n": len(instance["sizes"]),
        "Capacity": instance["capacity"],
        "Best Known": instance["best_known"],
        "ACO Bins": num_bins,
        "Gap": num_bins - instance["best_known"],
        "Unused": unused,
        "Runtime (s)": round(runtime, 4)
    })
    print(f"Processed {instance['name']}")
    
    # Save plots
    aco.plot_convergence(f"plots/{instance['name']}_convergence.png")
    aco.plot_load_distribution(f"plots/{instance['name']}_loads.png")
    aco.plot_item_size_distribution(f"plots/{instance['name']}_item_sizes.png")

# Print table
print("\nResults Table:")
print("| Instance | n | Capacity | Best Known | ACO Bins | Gap | Unused | Runtime (s) |")
print("|----------|---|----------|------------|----------|-----|--------|-------------|")
for res in results:
    print(f"| {res['Instance']} | {res['n']} | {res['Capacity']} | {res['Best Known']} | {res['ACO Bins']} | {res['Gap']} | {res['Unused']} | {res['Runtime (s)']} |")

# Additional plots
# Gap bar plot
plt.figure()
gaps = [res['Gap'] for res in results]
plt.bar([res['Instance'] for res in results], gaps)
plt.xlabel('Instance')
plt.ylabel('Gap to Best Known')
plt.title('Gap to Optimal')
plt.xticks(rotation=45)
plt.savefig('plots/gaps.png')
plt.close()

# Runtime bar plot
plt.figure()
runtimes = [res['Runtime (s)'] for res in results]
plt.bar([res['Instance'] for res in results], runtimes)
plt.xlabel('Instance')
plt.ylabel('Runtime (s)')
plt.title('Runtime Comparison')
plt.xticks(rotation=45)
plt.savefig('plots/runtimes.png')
plt.close()

# You can add more like unused space, etc.
# For better results on larger instances, increase max_iter or num_ants, but runtime increases.
# Optional: run multiple times and average metrics.

Processed u120_00
Processed u250_00
Processed u500_00
Processed u1000_00
Processed t60_00
Processed t120_00
Processed t249_00
Processed t501_00

Results Table:
| Instance | n | Capacity | Best Known | ACO Bins | Gap | Unused | Runtime (s) |
|----------|---|----------|------------|----------|-----|--------|-------------|
| u120_00 | 120 | 150 | 48 | 49 | 1 | 272.0 | 4.3808 |
| u250_00 | 250 | 150 | 99 | 100 | 1 | 217.0 | 10.9178 |
| u500_00 | 500 | 150 | 198 | 201 | 3 | 513.0 | 39.1729 |
| u1000_00 | 1000 | 150 | 399 | 403 | 4 | 686.0 | 125.4144 |
| t60_00 | 60 | 100.0 | 20 | 23 | 3 | 299.9999999999999 | 1.1761 |
| t120_00 | 120 | 100.0 | 40 | 45 | 5 | 499.9999999999999 | 2.5807 |
| t249_00 | 249 | 100.0 | 83 | 94 | 11 | 1099.9999999999995 | 7.9534 |
| t501_00 | 501 | 100.0 | 167 | 190 | 23 | 2299.9999999999986 | 24.6679 |
